In [18]:
import pandas as pd

apfs_df = pd.read_csv(r'Initial_data/ac_power_from_storage.csv')
apts_df = pd.read_csv(r'Initial_data/ac_power_to_storage.csv')
gc_df = pd.read_csv(r'Initial_data/grid_consumption.csv')
gf_df = pd.read_csv(r'Initial_data/grid_feed.csv')
pp_df = pd.read_csv(r'Initial_data/pv_production.csv')


In [19]:
'''Преобразуем колонку time в тип времени'''

apfs_df['time'] = pd.to_datetime(apfs_df['time'])
apts_df['time'] = pd.to_datetime(apts_df['time'])
gc_df['time'] = pd.to_datetime(gc_df['time'])
gf_df['time'] = pd.to_datetime(gf_df['time'])
pp_df['time'] = pd.to_datetime(pp_df['time'])

'''Устанавливаем время в качестве индекса'''

apfs_df.set_index('time', inplace=True)
apts_df.set_index('time', inplace=True)
gc_df.set_index('time', inplace=True)
gf_df.set_index('time', inplace=True)
pp_df.set_index('time', inplace=True)


'''Преобразуем колонку значений в тип float'''

apfs_df['ac_power_from_storage'] = apfs_df['ac_power_from_storage'].astype(float)
apts_df['ac_power_to_storage'] = apts_df['ac_power_to_storage'].astype(float)
gc_df['grid_consumption'] = gc_df['grid_consumption'].astype(float)
gf_df['grid_feed'] = gf_df['grid_feed'].astype(float)
pp_df['pv_production'] = pp_df['pv_production'].astype(float)


In [9]:
print(apfs_df)

                           ac_power_from_storage
time                                            
2016-06-10 16:53:00+00:00                 7460.0
2016-06-10 16:58:00+00:00                 5890.0
2016-06-10 17:03:00+00:00                 5870.0
2016-06-10 17:08:00+00:00                 1160.0
2016-06-10 17:13:00+00:00                 8410.0
...                                          ...
2023-09-11 14:50:30+00:00                    0.0
2023-09-11 14:55:30+00:00                    0.0
2023-09-11 15:00:30+00:00                    0.0
2023-09-11 15:05:30+00:00                    0.0
2023-09-11 15:10:30+00:00                    0.0

[622416 rows x 1 columns]


In [20]:
'''Группируем время чтобы получить равные промежутки'''

time_group = '5T' # интервал группировки 5 минут

g_apfs_df = apfs_df.resample('5T').sum()
g_apts_df = apts_df.resample('5T').sum()
g_gc_df = gc_df.resample('5T').sum()
g_gf_df = gf_df.resample('5T').sum()
g_pp_df = pp_df.resample('5T').sum()


In [22]:
print(g_pp_df)

                           pv_production
time                                    
2016-06-10 16:50:00+00:00        20832.8
2016-06-10 16:55:00+00:00        21650.2
2016-06-10 17:00:00+00:00        16588.1
2016-06-10 17:05:00+00:00        21694.7
2016-06-10 17:10:00+00:00        14891.5
...                                  ...
2023-09-11 14:50:00+00:00        19804.1
2023-09-11 14:55:00+00:00        19820.1
2023-09-11 15:00:00+00:00        25236.2
2023-09-11 15:05:00+00:00        25651.4
2023-09-11 15:10:00+00:00        24535.1

[762893 rows x 1 columns]


In [23]:
start_date = '2016-06-10 16:50:00+00:00'
end_date = '2023-09-12 00:00:00+00:00'

columns = ['time', 'P_l']
df_P_l = pd.DataFrame(columns=columns)

iteration_date = start_date
time_delta = pd.Timedelta(minutes=5)

while iteration_date < end_date:
    P_pv = g_pp_df['pv_production'][iteration_date]
    P_fu = g_gc_df['grid_consumption'][iteration_date]
    P_fs = g_apfs_df['ac_power_from_storage'][iteration_date]
    P_tu = g_gf_df['grid_feed'][iteration_date]
    P_ts = g_apts_df['ac_power_from_storage'][iteration_date]
    
    P_l = P_pv + P_fu + P_fs - (P_tu + P_ts)
    
    df_P_l = df_P_l.append({'time': iteration_date , 'P_l': P_l}, ignore_index=True)
    
    iteration_date += time_delta

KeyError: 'ac_power_from_storage'